#Exercise 1: Train a classifer to predict the output lable of data in test.csv file.




1.   Write DataLoaders
2.   Train DNN
3.   plot loss values
4.   Show a few samples of the train and test set
5.   provide the labels of the test_set in a file labelled Submission.csv. Submission.cvs should have a column called labels which should contain the labels of all the images




Note: The unpickle code will produce a dict (which have images and labels etc)


Take special care of reading images and make sure the images are indeed correct. You will have to plot the image to see it.

The images have dimensions: (32,32,3)

In [ ]:
!wget -O data.zip https://www.dropbox.com/s/bcrs90kyytjph42/cifar-10-batches-py.zip?dl=0
!unzip data.zip

--2023-01-19 08:13:07--  https://www.dropbox.com/s/bcrs90kyytjph42/cifar-10-batches-py.zip?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.65.18, 2620:100:6021:18::a27d:4112
Connecting to www.dropbox.com (www.dropbox.com)|162.125.65.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/raw/bcrs90kyytjph42/cifar-10-batches-py.zip [following]
--2023-01-19 08:13:07--  https://www.dropbox.com/s/raw/bcrs90kyytjph42/cifar-10-batches-py.zip
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uca50fba61ffb838ae6b7d0de2fa.dl-eu.dropboxusercontent.com/cd/0/inline/B01KesQ2QXiRvxD24rWkJPGxhFCZVIP1NuQfxtvMoFQQDpuWW_opP2OuxXLacH60qc5JteN24vdr8yxzONRLRvvHVjuJ_Qu0iELDy6A_CjMdc8ZpMH7tbhadEBjn2MArb4mramQumt1U-DirD1nqvByrh4HrcPVYGrxqppyig10uZA/file# [following]
--2023-01-19 08:13:07--  https://uca50fba61ffb838ae6b7d0de2fa.dl-eu.dropboxusercontent.com/cd/0/inline/B01KesQ2QXiRvxD24rWkJPGxhFCZVIP1N

In [ ]:
#Notice that train data has 5 batches. below code can be used
#to read the train data
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

In [ ]:
!ls cifar-10-batches-py/

batches.meta  data_batch_2  data_batch_4  readme.html
data_batch_1  data_batch_3  data_batch_5  test.csv


In [ ]:
import numpy as np
import torch
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torchvision.models import resnet18
import torch.optim as optim
import matplotlib.pyplot as plt


In [ ]:
data1 = unpickle("cifar-10-batches-py/data_batch_1")
ims1 = data1[b"data"]
labels1 = data1[b"labels"]

data2 = unpickle("cifar-10-batches-py/data_batch_2")
ims2 = data2[b"data"]
labels2 = data2[b"labels"]

data3 = unpickle("cifar-10-batches-py/data_batch_3")
ims3 = data3[b"data"]
labels3 = data3[b"labels"]

data4 = unpickle("cifar-10-batches-py/data_batch_4")
ims4 = data4[b"data"]
labels4 = data4[b"labels"]

data5 = unpickle("cifar-10-batches-py/data_batch_5")
ims5 = data5[b"data"]
labels5 = data5[b"labels"]

In [ ]:
images = np.concatenate([ims1,ims2, ims3, ims4, ims5], axis =0)
labels = np.concatenate([labels1, labels2, labels3, labels4, labels5], axis =0)

In [ ]:
labels.shape

(50000,)

In [ ]:
class MyDataset(Dataset):
  def __init__(self, data, labels):
    super().__init__()
    self.data = data
    self.labels = labels


  def __len__(self):
    return self.data.shape[0]

  def __getitem__(self, idx):
    im = self.data[idx, :]
    im = torch.from_numpy(np.reshape(im, (3,32,32))/255).type(torch.FloatTensor)
                                                      

    return (im,self.labels[idx])
    

In [ ]:
dataset = MyDataset(images, labels)

In [ ]:
train_loader = DataLoader(dataset, batch_size = 64, shuffle = True)

In [ ]:
net = resnet18(pretrained = True)
net.fc = nn.Linear(in_features=512, out_features=10, bias=True)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

In [ ]:
device = torch.device("cuda:0")
num_epochs = 10
lr = 1e-4

net = net.to(device)
opt = optim.Adam(net.parameters(), lr = lr)
ls = []
for i in range(num_epochs):
  total_loss = 0
  for batch in train_loader:
    X,y = batch[0].to(device), batch[1].to(device)
    logits = net(X)
    loss = F.cross_entropy(logits, y)
    opt.zero_grad()
    loss.backward()
    opt.step()
    total_loss+=loss.item()
  ls.append(total_loss)


plt.plot(ls)



In [ ]:
loss

tensor(2.6096, device='cuda:0', grad_fn=<NllLossBackward0>)